# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [1]:
from applets.core import get_connection
from applets.memorycard import MemoryCard

In [2]:
connection = get_connection()
if connection is None:
    print("Failed to open a connection. No cardreaders?")

## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [3]:
atr=bytes(connection.getATR())
print(atr.hex())

3bdc18ff8191fe1fc38073c821136605036351000250


## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [4]:
app = MemoryCard(connection)
app.select()

b''

In [5]:
# get random data without secure communication
app.get_random().hex()

'71cba3106d71ef6c59f72c6ab2f7d15dc373d66c229a6fc0321888adc62a8eec'

## 1.4. Establish secure communication

In [6]:
app.establish_secure_channel() # mode="se" and mode="ee" available, "se" by default

In [7]:
app.is_secure_channel_open

True

# 2. Start talking to the card securely

In [8]:
# sends encrypted data to the card, responce has the same data
app.echo(b'ping')

b'ping'

In [9]:
# get 32 bytes of randomness over secure channel
app.get_random_sc()

b'\xa2=\xfd\xbd\xe2K\rS\x1f\x85\x8b\x90\xd6\x1a\xd4\xe6\xee\xf7\x84\xce\x98EhX\xf6\x14\x1ay\xc9\xbf \xbf'

In [10]:
# get status
app.secure_request(b'\x03\x00').hex()

'0a0a00'

In [11]:
# unlock with pin, or set pin. Pin can be anything up to 32 bytes. Want more? - hash it.
app.secure_request(b'\x03\x01'+b'q')

b''

In [12]:
# if you are setting new pin - call it twice - first to set pin, second - to unlock card.
app.secure_request(b'\x03\x01'+b'q')

b''

In [13]:
# change pin
app.secure_request(b'\x03\x03'+b'\x01'+b'q'+b'\x06'+b'passwd')

b''

In [14]:
# unlock with new pin
app.secure_request(b'\x03\x01'+b'passwd')

b''

In [15]:
# get data over secure channel
app.get_secret_data()

b'Memory cards are not safu so what?'

In [16]:
# set data over secure channel
mnemonic = b'universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe urge'
app.put_secret_data(mnemonic)

b''

In [17]:
# check the new data is stored now
app.get_secret_data()

b'universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe universe urge'

In [18]:
# lock card
app.secure_request(b'\x03\x02')

b''

In [19]:
app.close_secure_channel()

In [20]:
connection.disconnect()